# State of the Climate 2025: Antarctic surface mass balance

ATL15 surface height change for 2025

In [ ]:
SOTC_YEAR = 2025

In [ ]:
import pathlib
import itertools

import numpy as np
import icesat2_smb as is2smb
import xarray as xr
import rasterio as rio

import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
# autoreload of modules
%load_ext autoreload
%autoreload 2

In [ ]:
# Data parent directory
dir_data = pathlib.Path('/mnt/c/Users/taryn/Data')

# ICESat-2 data (Antarctica)
# -- ATL15 Gridded Land Ice Height Change
# -- Info and latest version: https://nsidc.org/data/atl15
f_ATL15 = list(dir_data.joinpath('ATL15').glob('ATL15_A*.nc'))

# ATL15 grounded ice coverage mask
# -- Rasterized version of mask on IMBIE website, provided by Tyler Sutterley
f_mask = list(dir_data.joinpath('ATL15/ATL15_AA_10km_003_IMBIE2_v1.6').glob('*.nc'))

# ERA5 SMB anomaly
f_ERA5 = dir_data.joinpath('SOTC_2025/era5_SMB_2025_Anoms_Monthly.nc')

In [ ]:
atl15_spatial_resolution = f_ATL15[0].stem.split('_')[3]

# Prepare data

## Load data

In [ ]:
# ATL15
ATL15_groups = [
    'delta_h', # height change relative to the ATL14 DEM [m]
    'dhdt_lag1', # quarterly height change rate [m/a]
    'dhdt_lag4', # annual height change rate [m/a]
]
atl15_original = is2smb.io.read_ATL15(
    file=f_ATL15,
    group=ATL15_groups,
    decimal_years=True,
)
atl15_original.name = "ATL15"

# Grounded ice mask
# -- rename to avoid conflict with ATL15 dataset names
grounded_mask = is2smb.io.read_mask(f_mask)
grounded_mask = grounded_mask.rename({'polar_stereographic': 'Polar_Stereographic'})

# ERA5 SMB anomaly
era5_original = xr.open_dataset(f_ERA5, decode_coords=True)
era5_original = era5_original.rio.write_crs(4326)

## Regrid model data to match ATL15

In [ ]:
# ERA5
era5 = is2smb.spatial.interpolate_grid(
    data=era5_original,
    regrid_like=atl15_original.delta_h.delta_h,
)

## Apply grounded ice mask

In [ ]:
# ATL15
atl15 = atl15_original.copy()
atl15 = atl15.map_over_datasets(
    func=is2smb.spatial.apply_mask,
    kwargs=dict(mask=grounded_mask)
)

# ERA5
era5 = is2smb.spatial.apply_mask(
    data=era5,
    mask=grounded_mask
)

# ATL15

## Unmodified heights

### Height anomaly (`delta_h`)

In [ ]:
data = atl15.delta_h.sel(time=slice(SOTC_YEAR, None))
times = data.time.values
units = 'm'

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times),
    figsize=(10,4),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax)

props = dict(
    cmap='RdBu',
    norm=colors.CenteredNorm(vcenter=0, halfrange=1)
)

for i, t in enumerate(times):
    h = data.delta_h.sel(time=t).plot(
        ax=ax[i],
        add_colorbar=False,
        **props
    )
    ax[i].set_frame_on(False)
    ax[i].set_title(t)

fig.colorbar(h, ax=ax[-1], extend='both', label=f'height anomaly [${units}$]')
fig.suptitle(
    f"""\
    Quarterly height relative to datum surface (1 January 2020)
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

### Quarterly height change rate (`dhdt_lag1`)

In [ ]:
# Plot: maps

dt = 0.25
data = atl15.dhdt_lag1.sel(time=slice(SOTC_YEAR - dt/2, None))
times = data.time.values
units = r"m\ a^{-1}"

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times),
    figsize=(10,4),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax)

props = dict(
    cmap='RdBu',
    norm=colors.CenteredNorm(vcenter=0, halfrange=1)
)

for i, t in enumerate(times):
    h = data.dhdt.sel(time=t).plot(
        ax=ax[i],
        add_colorbar=False,
        **props
    )
    ax[i].set_frame_on(False)
    ttxt = f'{t - dt/2} to {t + dt/2}'
    ax[i].set_title(ttxt)

fig.colorbar(h, ax=ax[-1], extend='both', label=f"height change rate [${units}$]")
fig.suptitle(
    f"""\
    Quarterly rate of height change
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

In [ ]:
# Plot: time series of integrated volume change
vol_km3 = (data.dhdt * data.ice_area) / 1E9
integrated_volume_rate = vol_km3.sum(
    skipna=True,
    min_count=1,
    dim=['x', 'y']
)

fig, ax = plt.subplots(
    figsize=(5,2),
)
integrated_volume_rate.plot(
    ax=ax, 
    marker='.'
)
ax.set_xlabel('time')
ax.set_ylabel(r'volume flux [$km^3\ a^{-1}$]')
ax.set_title('Integrated quarterly volume flux')
plt.show()

### Annual height change rate (`dhdt_lag4`)

In [ ]:
dt = 1
data = atl15.dhdt_lag4.sel(time=slice(SOTC_YEAR - dt/2, None))
times = data.time.values
units = r"m\ a^{-1}"

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times),
    figsize=(10,4),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax)

props = dict(
    cmap='RdBu',
    norm=colors.CenteredNorm(vcenter=0, halfrange=0.25)
)

for i, t in enumerate(times):
    h = data.dhdt.sel(time=t).plot(
        ax=ax[i],
        add_colorbar=False,
        **props
    )
    ax[i].set_frame_on(False)
    ttxt = f'{t - dt/2} to {t + dt/2}'
    ax[i].set_title(ttxt)

fig.colorbar(h, ax=ax[-1], extend='both', label=f"height change rate [${units}$]")
fig.suptitle(
    f"""\
    Annual rate of height change
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

In [ ]:
# Plot: time series of integrated volume change
vol_km3 = (data.dhdt * data.ice_area) / 1E9
integrated_volume_rate = vol_km3.sum(
    skipna=True,
    min_count=1,
    dim=['x', 'y']
)

fig, ax = plt.subplots(
    figsize=(5,2),
)
integrated_volume_rate.plot(
    ax=ax, 
    marker='.'
)
ax.set_xlabel('time')
ax.set_ylabel(r'volume flux [$km^3\ a^{-1}$]')
ax.set_title('Integrated annual volume flux')
plt.show()

## Detrended heights

### Detrend height data

In [ ]:
# Detrend quarterly and annual dhdt (remove the mean) for ATL15
for src, L in itertools.product([atl15], ['dhdt_lag1', 'dhdt_lag4']):
    src[L]['dhdt'] = is2smb.spatial.detrend_da(
        da=src[L].dhdt,
        dim='time',
        degree=0
    )

trend = atl15_original - atl15

### Quarterly height change rate (`dhdt_lag1`)

In [ ]:
# Plot: trend and quarterly maps

dt = 0.25
lag = 'dhdt_lag1'

data = atl15[lag].sel(time=slice(SOTC_YEAR - dt/2, None))
times = data.time.values
units = r"m\ a^{-1}"
cmap='RdBu'

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times)+1,
    figsize=(12,3.25),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)

# plot trend
trend[lag].dhdt.isel(time=-1).plot(
    ax=ax[0],
    cmap=cmap,
    norm=colors.CenteredNorm(vcenter=0, halfrange=0.25),
    cbar_kwargs=dict(
        extend='both',
        label=f"height change rate [${units}$]",
    ),
)
ttxt = f'Trend, {atl15[lag].time.values[0] - dt/2} to {atl15[lag].time.values[-1] + dt/2}'
ax[0].set_title(ttxt)

for i, t in enumerate(times):
    h = data.dhdt.sel(time=t).plot(
        ax=ax[i+1],
        cmap=cmap,
        norm=colors.CenteredNorm(vcenter=0, halfrange=1),
        add_colorbar=False,
    )
    ttxt = f'dhdt, {t - dt/2} to {t + dt/2}'
    ax[i+1].set_title(ttxt)

# prettify
fig.colorbar(h, ax=ax[-1], extend='both', label=f"height change rate [${units}$]")
fig.suptitle(
    f"""\
    Quarterly rate of height change (detrended)
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

### Annual height change rate (`dhdt_lag4`)

In [ ]:
# Plot: trend and annual maps

dt = 1
lag = 'dhdt_lag4'

data = atl15[lag].sel(time=slice(SOTC_YEAR - dt/2, None))
times = data.time.values
units = r"m\ a^{-1}"

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times)+1,
    figsize=(12,3.5),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)

props = dict(
    cmap='RdBu',
    norm=colors.CenteredNorm(vcenter=0, halfrange=0.25)
)

# plot trend
trend[lag].dhdt.isel(time=-1).plot(
    ax=ax[0],
    add_colorbar=False,
    **props
)
ttxt = f'Trend, {atl15[lag].time.values[0] - dt/2} to {atl15[lag].time.values[-1] + dt/2}'
ax[0].set_title(ttxt)

for i, t in enumerate(times):
    h = data.dhdt.sel(time=t).plot(
        ax=ax[i+1],
        add_colorbar=False,
        **props
    )
    ttxt = f'dhdt, {t - dt/2} to {t + dt/2}'
    ax[i+1].set_title(ttxt)

# prettify
fig.colorbar(h, ax=ax[-1], extend='both', label=f"height change rate [${units}$]")
fig.suptitle(
    f"""\
    Annual rate of height change (detrended)
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

# ERA5

In [ ]:
h = era5.SMB_anom_pct.plot(
    col='time',
    col_wrap=3,
    norm=colors.CenteredNorm(vcenter=0, halfrange=100),
)

for i, ax in enumerate(h.axs.flat):
    ax.set_aspect('equal')
#     date = era.time.isel(time=i).values
#     ax.set_title(date)

## Group by quarters

In [ ]:
qdates = [f'{SOTC_YEAR}-01-01', f'{SOTC_YEAR}-04-01', f'{SOTC_YEAR}-07-01', f'{SOTC_YEAR}-10-01', f'{SOTC_YEAR+1}-01-01']
quarters = [np.datetime64(x) for x in qdates]
decdates = np.arange(SOTC_YEAR+0.125, SOTC_YEAR+1.125, step=0.25)
era5_quarters = era5.groupby_bins(group='time', bins=quarters, labels=decdates).mean()
era5_quarters = era5_quarters.rename(dict(time_bins='time'))
era5_quarters

In [ ]:
era5_quarters.SMB_anom_pct.plot(col='time', cmap='RdBu', norm=colors.CenteredNorm(vcenter=0, halfrange=100))

need to think about how to do this properly/what this means. I'm not convinced that the mean of the anomalies is appropriate.

In [ ]:
ratio = atl15.dhdt_lag1.dhdt.sel(time=slice(2025,None)) / era5_quarters.SMB_anom_pct.isel(time=slice(None,2))
# ratio.plot(col='time', cmap='RdBu', norm=colors.CenteredNorm(vcenter=0, halfrange=))
ratio.plot(col='time', cmap='RdBu', robust=True)